In [ ]:
%%writefile prog.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cusolverDn.h>

int main() {
    const int n = 3; // size of the matrix
    const int nrhs = 1; // number of right-hand sides

    // example matrix - macierz zapisujemy {[0,0], [0,1], [0,2], [1,0], [1,1], [1,2] ,[2,0], [2,1], [2,2]}  -kolumnami
    float A[n*n] = {2, 4, -2, 1, -6, 7, 1, 0, 2}; 
    float B[n*nrhs] = {5,-2,9};

    // allocate memory on the device
    float *dA, *dB, *dX;
    int *dipiv; // pivoting information

    cudaMalloc((void**)&dA, n * n * sizeof(float));
    cudaMalloc((void**)&dB, n * nrhs * sizeof(float));
    cudaMalloc((void**)&dX, n * nrhs * sizeof(float));
    cudaMalloc((void**)&dipiv, n * sizeof(int));

    // copy input matrices to the device
    cudaMemcpy(dA, A, n * n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(dB, B, n * nrhs * sizeof(float), cudaMemcpyHostToDevice);

    // create cusolverDn handle
    cusolverDnHandle_t handle;
    cusolverDnCreate(&handle);

    // factorize and solve the linear system
    int niter, dinfo;
    size_t workspace_size;
    cusolverStatus_t status = cusolverDnSSgesv_bufferSize(handle, n, nrhs, dA, n, dipiv, dB, n, dX, n, NULL, &workspace_size);

    if (status != CUSOLVER_STATUS_SUCCESS) {
        printf("Error querying workspace size: %d\n", status);
        return 1;
    }

    status = cusolverDnSSgesv(handle, n, nrhs, dA, n, dipiv, dB, n, dX, n, NULL, workspace_size, &niter, &dinfo);

    printf("%d %long %d %d\n", status, workspace_size, dinfo, niter);
    if (status != CUSOLVER_STATUS_SUCCESS) {
        printf("Error solving linear system: %d\n", status);
        return 1;
    }

    if (dinfo != 0) {
        printf("Error solving linear system: dinfo = %d\n", dinfo);
        return 1;
    }

    // copy the solution back to the host
    float X[n*nrhs];
    cudaMemcpy(X, dX, n * nrhs * sizeof(float), cudaMemcpyDeviceToHost);

    // print the solution
    printf("Solution:\n");
    for (int i = 0; i < n; i++) {
        printf("%f\n", X[i]);
    }

    // free memory
    cudaFree(dA);
    cudaFree(dB);
    cudaFree(dX);
    cudaFree(dipiv);
    cusolverDnDestroy(handle);

    return 0;
}


Writing prog.cu


In [ ]:
!nvcc -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include -lcublas -lcusolver -arch=sm_35 -Wno-deprecated-gpu-targets prog.cu

In [ ]:
!./a.out

7 74000ng 32603 0
Error solving linear system: 7


In [ ]:
%%writefile prog.cu
// This program computes a simple version of matrix multiplication
// By: Nick from CoffeeBeforeArch

#include <algorithm>
#include <cassert>
#include <cstdlib>
#include <functional>
#include <iostream>
#include <vector>

using std::cout;
using std::generate;
using std::vector;

__global__ void matrixMul(const int *a, const int *b, int *c, int N) {
  // Compute each thread's global row and column index
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  // Iterate over row, and down column
  c[row * N + col] = 0;
  for (int k = 0; k < N; k++) {
    // Accumulate results for a single element
    c[row * N + col] += a[row * N + k] * b[k * N + col];
  }
}

// Check result on the CPU
void verify_result(vector<int> &a, vector<int> &b, vector<int> &c, int N) {
  // For every row...
  for (int i = 0; i < N; i++) {
    // For every column...
    for (int j = 0; j < N; j++) {
      // For every element in the row-column pair
      int tmp = 0;
      for (int k = 0; k < N; k++) {
        // Accumulate the partial results
        tmp += a[i * N + k] * b[k * N + j];
      }

      // Check against the CPU result
      assert(tmp == c[i * N + j]);
    }
  }
}

int main() {
  // Matrix size of 1024 x 1024;
  int N = 1 << 10;

  // Size (in bytes) of matrix
  size_t bytes = N * N * sizeof(int);

  // Host vectors
  vector<int> h_a(N * N);
  vector<int> h_b(N * N);
  vector<int> h_c(N * N);

  // Initialize matrices
  generate(h_a.begin(), h_a.end(), []() { return rand() % 100; });
  generate(h_b.begin(), h_b.end(), []() { return rand() % 100; });

  // Allocate device memory
  int *d_a, *d_b, *d_c;
  cudaMalloc(&d_a, bytes);
  cudaMalloc(&d_b, bytes);
  cudaMalloc(&d_c, bytes);

  // Copy data to the device
  cudaMemcpy(d_a, h_a.data(), bytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b.data(), bytes, cudaMemcpyHostToDevice);

  // Threads per CTA dimension
  int THREADS = 32;

  // Blocks per grid dimension (assumes THREADS divides N evenly)
  int BLOCKS = N / THREADS;

  // Use dim3 structs for block  and grid dimensions
  dim3 threads(THREADS, THREADS);
  dim3 blocks(BLOCKS, BLOCKS);

  // Launch kernel
  matrixMul<<<blocks, threads>>>(d_a, d_b, d_c, N);

  // Copy back to the host
  cudaMemcpy(h_c.data(), d_c, bytes, cudaMemcpyDeviceToHost);

  // Check result
  verify_result(h_a, h_b, h_c, N);

  cout << "COMPLETED SUCCESSFULLY\n";

  // Free memory on device
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}

Overwriting prog.cu


In [ ]:
!nvcc -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include -lcublas -lcusolver -arch=sm_35 -Wno-deprecated-gpu-targets prog.cu
!./a.out